# Colorer les résultats de SWIM
Pour voir la distribution des résultats de la génération par SWIM, les cases sont colorées en fonction de leur valeur :
- bleu pour les cases dans l'échantillon de départ
- vert pour les cases bien générées
- rouge pour les erreurs
- orange pour les sur-générations
- jaune pour les cases non-générées
- blanc pour les cases vides dans le Gold

Les résultats *paradigmes-Swim?.csv* sont pris dans le répertoire où ils ont été calculés. Les annotations sont écrites dans des fichiers .xlsx correspondants dans le répertoire *HDR/Longitudinales/XLS*

In [1]:
import pandas as pd
import numpy as np
import yaml,re,pyperclip
import EspacesThematiques as ET
from EspacesThematiques import *
from FrenchPhonology import makeFrench,setNeutralisation

debug=False
import datetime,glob
def dateheure(f="%y%m%d%H%M"):
    return datetime.datetime.utcnow().strftime(f) # %y%m%d%H%M

S Z N J ê ô â r E H O 6 9 2


In [2]:
cases=etCols
print dateheure("%H:%M")
phonologicalMap="NS"
neutralise=setNeutralisation(phonologicalMap)

18:30


In [3]:
repCSV="/Users/gilles/pCloud Drive/FOD/GB/2015-Data/Longitudinales/"
repXLS="/Users/gilles/ownCloud/Recherche/Boye/HDR/Memoire/Longitudinales/PhonoMap-X-NS/"
fichiersPar=glob.glob(repCSV+"*paradigmes*.csv")
fichiersOG=glob.glob(repCSV+"*overGeneration*.csv")
fichiersFP=glob.glob(repCSV+"*diffForms*.csv")
fichiersUG=glob.glob(repCSV+"*underGeneration*.csv")

In [4]:
def findFichier(patron,fichiers):
    candidats=[]
    for sample in fichiers:
        m=re.match(patron,sample)
        if m:
            candidats.append(m.group(1))
    if len(candidats)==1:
        return candidats[0]
    else:
        print "PB pas de nom unique correspondant",candidats

In [5]:
def colorCells(row):
    result=[""]
    lexeme=row["lexeme"]
    for case in casesPar:
        if all(row[case]==dfPar.loc[dfPar.lexeme==lexeme,case]):
            result.append('background-color: blue')
        elif u"%s-%s"%(lexeme,case) in dfFP["lexeme-case"].tolist():
            formeBrute=dfFP[dfFP["lexeme-case"]==u"%s-%s"%(lexeme,case)]["value"].values[0]
            formeGold=dfFP[dfFP["lexeme-case"]==u"%s-%s"%(lexeme,case)]["right"].values[0]
            if "," not in formeGold:
#                print formeBrute,formeGold
                if makeFrench(formeBrute,neutralise)!=makeFrench(formeGold,neutralise):
                    result.append('background-color: red')
                else:
                    result.append('background-color: #90EE90')
            else:
                if makeFrench(formeBrute,neutralise) in formeGold.split(","):
                    result.append('background-color: #FF00FF')
                else:
                    result.append('background-color: #8B0000')                
        elif u"%s-%s"%(lexeme,case) in dfOG["lexeme-case"].tolist():        
            result.append('background-color: orange')
        elif u"%s-%s"%(lexeme,case) in dfUG["lexeme-case"].tolist():        
            result.append('background-color: yellow')
        elif row[case]==row[case]:
            result.append('background-color: green')
        else:
            result.append("")
    return result

<a id="lecture"></a>
Liens :
- [lecture erreurs](#lecture)
- [Excel](#excel)

### Lecture fichiers erreurs

In [6]:
casesTypes=["-X-Morphomes"]#,"-X"]
echantillons=["%02d"%n for n in range(29,130)]
etapes=["-Swim2"]#,"-Swim1"]
for casesType in casesTypes:
    for numeroEchantillon in echantillons:
        print dateheure("%H:%M"),
        print numeroEchantillon
        patron=ur"^.*/(.*Longitudinal-%s-.*-F\d+%s-%s%s\.csv)"%(numeroEchantillon,casesType,"paradigmes","")
        nomPar=findFichier(patron,fichiersPar)
        dfPar=pd.read_csv(repCSV+nomPar,sep=";",encoding="utf8")
        casesPar=[c for c in cases if c in dfPar.columns]
        dfPar=dfPar[["lexeme"]+casesPar]
        for etape in etapes:
            print etape,
            patron=patron=ur"^.*/(.*Longitudinal-%s-.*-F\d+%s-%s%s.*\.csv)"%(numeroEchantillon,casesType,"paradigmes",etape)
            nomRes=findFichier(patron,fichiersPar)
            dfRes=pd.read_csv(repCSV+nomRes,sep=";",encoding="utf8")
            dfRes=dfRes[["lexeme"]+casesPar]

            patron=ur"^.*/(.*Longitudinal-%s-.*-F\d+%s-%s%s.*\.csv)"%(numeroEchantillon,casesType,"overGeneration",etape)
            nomOG=findFichier(patron,fichiersOG)

            patron=ur"^.*/(.*Longitudinal-%s-.*-F\d+%s-%s%s.*\.csv)"%(numeroEchantillon,casesType,"diffForms",etape)
            nomFP=findFichier(patron,fichiersFP)

            patron=ur"^.*/(.*Longitudinal-%s-.*-F\d+%s-%s%s.*\.csv)"%(numeroEchantillon,casesType,"underGeneration",etape)
            nomUG=findFichier(patron,fichiersUG)

            dfOG=pd.read_csv(filepath_or_buffer=repCSV+nomOG,encoding="utf8")
            dfFP=pd.read_csv(filepath_or_buffer=repCSV+nomFP,encoding="utf8")
            dfUG=pd.read_csv(filepath_or_buffer=repCSV+nomUG,encoding="utf8")

            dfOG["lexeme"]=dfOG["lexeme-case"].str.rsplit("-",1).str[0]
            dfOG["case"]=dfOG["lexeme-case"].str.rsplit("-",1).str[1]
            dfFP["lexeme"]=dfFP["lexeme-case"].str.rsplit("-",1).str[0]
            dfFP["case"]=dfFP["lexeme-case"].str.rsplit("-",1).str[1]
            dfUG["lexeme"]=dfUG["lexeme-case"].str.rsplit("-",1).str[0]
            dfUG["case"]=dfUG["lexeme-case"].str.rsplit("-",1).str[1]

            dfResCol=dfRes.style.apply(colorCells,axis=1)
            nomResCol=repXLS+nomRes.replace("-X-","-%s-"%phonologicalMap) # bug avec .xls
            nomResCol=nomResCol.replace(".csv","-Annot.xlsx")
            dfResCol.to_excel(nomResCol)
        print dateheure("%H:%M")
        print


18:30 29
-Swim2 18:34

18:34 30
-Swim2 18:39

18:39 31
-Swim2 18:44

18:44 32
-Swim2 18:48

18:48 33
-Swim2 18:54

18:54 34
-Swim2 18:59

18:59 35
-Swim2 19:05

19:05 36
-Swim2 19:11

19:11 37
-Swim2 19:18

19:18 38
-Swim2 19:23

19:23 39
-Swim2 19:28

19:28 40
-Swim2 19:33

19:33 41
-Swim2 19:38

19:38 42
-Swim2 19:44

19:44 43
-Swim2 19:49

19:49 44
-Swim2 19:54

19:54 45
-Swim2 19:59

19:59 46
-Swim2 20:04

20:04 47
-Swim2 20:09

20:09 48
-Swim2 20:15

20:15 49
-Swim2 20:21

20:21 50
-Swim2 20:26

20:26 51
-Swim2 20:31

20:31 52
-Swim2 20:36

20:36 53
-Swim2 20:41

20:41 54
-Swim2 20:46

20:46 55
-Swim2 20:51

20:51 56
-Swim2 20:56

20:56 57
-Swim2 21:01

21:01 58
-Swim2 21:06

21:06 59
-Swim2 21:11

21:11 60
-Swim2 21:15

21:15 61
-Swim2 21:20

21:20 62
-Swim2 21:25

21:25 63
-Swim2 21:31

21:31 64
-Swim2 21:36

21:36 65
-Swim2 21:41

21:41 66
-Swim2 21:45

21:45 67
-Swim2 21:50

21:50 68
-Swim2 21:55

21:55 69
-Swim2 22:00

22:00 70
-Swim2 22:05

22:05 71
-Swim2 22:10

22:10 72
-S